<a href="https://colab.research.google.com/github/lihui1625/deepleaning/blob/main/d2l_P08_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install d2l

In [ ]:
import collections
import re
from d2l import torch as d2l

In [ ]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a')

In [ ]:
def read_time_machine():
  with open(d2l.download('time_machine'),'r') as f:
    lines = f.readlines()
  return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

In [ ]:
lines = read_time_machine()
len(lines)

3221

In [ ]:
lines[:11]

['the time machine by h g wells',
 '',
 '',
 '',
 '',
 'i',
 '',
 '',
 'the time traveller for so it will be convenient to speak of him',
 'was expounding a recondite matter to us his grey eyes shone and',
 'twinkled and his usually pale face was flushed and animated the']

In [ ]:
(lines[10]).split()

['twinkled',
 'and',
 'his',
 'usually',
 'pale',
 'face',
 'was',
 'flushed',
 'and',
 'animated',
 'the']

In [ ]:
list((lines[10]))

['t',
 'w',
 'i',
 'n',
 'k',
 'l',
 'e',
 'd',
 ' ',
 'a',
 'n',
 'd',
 ' ',
 'h',
 'i',
 's',
 ' ',
 'u',
 's',
 'u',
 'a',
 'l',
 'l',
 'y',
 ' ',
 'p',
 'a',
 'l',
 'e',
 ' ',
 'f',
 'a',
 'c',
 'e',
 ' ',
 'w',
 'a',
 's',
 ' ',
 'f',
 'l',
 'u',
 's',
 'h',
 'e',
 'd',
 ' ',
 'a',
 'n',
 'd',
 ' ',
 'a',
 'n',
 'i',
 'm',
 'a',
 't',
 'e',
 'd',
 ' ',
 't',
 'h',
 'e']

In [ ]:
def tokenize(lines, token='word'):
  if token == 'word':
    return [line.split() for line in lines]
  elif token == 'char':
    return [list(line) for line in lines]
  else:
    print("Error: ", token)

In [ ]:
tokens = tokenize(lines)
tokens[:11]

[['the', 'time', 'machine', 'by', 'h', 'g', 'wells'],
 [],
 [],
 [],
 [],
 ['i'],
 [],
 [],
 ['the',
  'time',
  'traveller',
  'for',
  'so',
  'it',
  'will',
  'be',
  'convenient',
  'to',
  'speak',
  'of',
  'him'],
 ['was',
  'expounding',
  'a',
  'recondite',
  'matter',
  'to',
  'us',
  'his',
  'grey',
  'eyes',
  'shone',
  'and'],
 ['twinkled',
  'and',
  'his',
  'usually',
  'pale',
  'face',
  'was',
  'flushed',
  'and',
  'animated',
  'the']]

In [ ]:
def count_corpus(tokens):
  if len(tokens) == 0 or isinstance(tokens[0], list):
    tokens = [token for line in tokens for token in line ]
  return collections.Counter(tokens)

In [ ]:
cc = count_corpus(tokens)
cc

In [ ]:
class Vocab:

  def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
    if tokens is None:
      tokens = []
    if reserved_tokens is None:
      reserved_tokens = []
    counter = count_corpus(tokens)
    self._token_freqs = sorted(counter.items(), key=lambda x:x[1], reverse=True)
    self.idx_to_token = ['<unk>'] + reserved_tokens
    self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
    for token, freq in self._token_freqs:
      if freq < min_freq:
        break
      if token not in self.token_to_idx:
        self.idx_to_token.append(token)
        self.token_to_idx[token] = len(self.idx_to_token) - 1


  def __len__(self):
    return len(self.idx_to_token)

  def __getitem__(self, tokens):
    if not isinstance(tokens, (list, tuple)):
      return self.token_to_idx.get(tokens, self.unk)
    return [self.__getitem__(token) for token in tokens]

  def to_tokens(self, indices):
    if not isinstance(indices, (list, tuple)):
      return self.idx_to_token[indices]
    return [self.idx_to_token[index] for index in indices]

  @property
  def unk(self):
    return 0

  @property
  def token_freqs(self):
    return self._token_freqs


In [ ]:
vocab = Vocab(tokens)

In [ ]:
list(vocab.token_to_idx.items())[:10]

[('<unk>', 0),
 ('the', 1),
 ('i', 2),
 ('and', 3),
 ('of', 4),
 ('a', 5),
 ('to', 6),
 ('was', 7),
 ('in', 8),
 ('that', 9)]

In [ ]:
for i in [0, 10]:
  print(f"text: {tokens[i]}")
  print(f"index: {vocab[tokens[i]]}")

text: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
index: [1, 19, 50, 40, 2183, 2184, 400]
text: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
index: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [ ]:
def load_corpus_time_machine(max_tokens=-1):
  lines = read_time_machine()
  tokens = tokenize(lines, 'char')
  vocab = Vocab(tokens)
  corpus = [vocab[token] for line in tokens for token in line]
  if max_tokens > 0:
    corpus = corpus[:max_tokens]
  return corpus, vocab


In [ ]:
corpus, vocab = load_corpus_time_machine()

In [ ]:
len(corpus), len(vocab)

(170580, 28)

In [ ]:
corpus[:10]

[3, 9, 2, 1, 3, 5, 13, 2, 1, 13]

In [ ]:
vocab.token_freqs

[(' ', 29927),
 ('e', 17838),
 ('t', 13515),
 ('a', 11704),
 ('i', 10138),
 ('n', 9917),
 ('o', 9758),
 ('s', 8486),
 ('h', 8257),
 ('r', 7674),
 ('d', 6337),
 ('l', 6146),
 ('m', 4043),
 ('u', 3805),
 ('c', 3424),
 ('f', 3354),
 ('w', 3225),
 ('g', 3075),
 ('y', 2679),
 ('p', 2427),
 ('b', 1897),
 ('v', 1295),
 ('k', 1087),
 ('x', 236),
 ('z', 144),
 ('j', 97),
 ('q', 95)]

In [ ]:
corpus, vocab = load_corpus_time_machine(1000)

In [ ]:
len(corpus), len(vocab)

(1000, 28)